# Imports

In [1]:
import os   # for path operations
import datetime

import tensorflow as tf
from tensorflow.python.client import device_lib

import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers.core import Activation, Dense, Flatten
from keras.optimizers import SGD
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import matplotlib.pyplot as plt

Using TensorFlow backend.


# Config GPU env

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
# Device check
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15916943652870412808
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3168377241
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17564788919795856927
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


# Def NN

## Training options

In [3]:
patience=40
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath="weights.hdf5", save_best_only=True, verbose =1)

Create a 1D convnet based on the practice lesson for time series analysis

In [4]:
def make_1d_convnet(window_size, filter_length, nb_input_series=1, nb_outputs=3,nb_filter=4):
    model = Sequential()
    model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu', 
                   input_shape=(window_size,nb_input_series)))
    model.add(MaxPooling1D())
    model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(nb_outputs,activation='linear'))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

Create input and output format from raw data

In [9]:
def evalute_timeseries(timeseries,yindex, window_size, filter_length, nb_filter , epochs):
    #timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T
    nb_samples, nb_series = timeseries.shape
    nb_inputs = nb_series - 11 #only 12 inputs (previous x,y,z position is an input now)
    #nb_outputs = nb_series - 9 #only 9 outputs
    nb_outputs=3
  
    model= make_1d_convnet(window_size=window_size, filter_length=filter_length, nb_input_series=nb_inputs,
                        nb_outputs=nb_outputs,nb_filter=nb_filter)
    model.summary()
    
    # def make_timeseries_instances
    timeseries = np.asarray(timeseries)
    assert 0 < window_size < timeseries.shape[0] , "Out of range 0 < {} < {}".format(window_size,timeseries.shape[0])
    # nb_input defines the number of input time series that we use to predict outputs
    X = np.atleast_3d(np.array([timeseries[start:start+window_size,:nb_inputs] for start in range(0,timeseries.shape[0]-window_size)]))
    # We have 3 output signal: x, y and z position
    # y = timeseries[window_size:,-9:-6]
    # We have 1 output signal: orient
    y = timeseries[window_size:,-11:-3]
  
    test_size = int(0.3  * nb_samples)
    valid_size = int(0.2 * nb_samples)
    X_train, X_valid, X_test = X[:-(test_size+valid_size),:], X[-(test_size+valid_size):-test_size,:], X[-test_size:,:]
    y_train, y_valid, y_test = y[:-(test_size+valid_size),:], y[-(test_size+valid_size):-test_size,:], y[-test_size:,:]
    
    # Standardize input variables for train, valid and test data
    for inpt in range(X.shape[1]):
        scaler = preprocessing.StandardScaler().fit(X_train[:,inpt])
        X_train[:,inpt] = scaler.transform(X_train[:,inpt])
        X_valid[:,inpt] = scaler.transform(X_valid[:,inpt])
        X_test[:,inpt] = scaler.transform(X_test[:,inpt])
    
    model.fit(X_train,y_train, epochs = epochs, batch_size = 16, validation_data = (X_valid, y_valid),
              verbose = 2, callbacks = [checkpointer,early_stopping])
  
    preds=model.predict(X_test)
    targets = y_test
    
    print(preds.shape)
    print(targets.shape)
    
    # Plot predicted x vs target x
    plt.figure(0)
    plt.plot(preds[:,0],color='green')
    plt.plot(targets[:,0],color='red')
    
    # Plot predicted z vs target z
    plt.figure(1)
    plt.plot(preds[:,2],color='blue')
    plt.plot(targets[:,2],color='yellow')
    
    # Plot the predicted route vs original route on x-z plane
    plt.figure(2)
    plt.plot(preds[:,0],preds[:,2])
    plt.plot(targets[:,0],targets[:,2])

# Load Data

In [10]:
lock_meas = '..\\..\\DATA\\RAW\\Measure_02' # Location folder of the measurement files
file_list = os.listdir(lock_meas)           # Make a list out of the files in the folder
nb_meas = len(file_list)/4                  # Every measurement contains 4 file (IMU, MoCap, Robotino, NFO)

meas_list = []                              # Make a list containing the independent measurements
for FILE_NAME in file_list:
    meas_list.append(FILE_NAME[0:22])       # The unique ID is the time, the first 22 character
meas_list = list(set(meas_list))

# Load in one favourite measurement block
meas_id = 4

# Syncronise the measurement files from MoCap and IMU
if meas_id>(nb_meas-1):
    meas = 0
meas_date = str(meas_list[meas_id])

# Load the IMU csv data file
imu_data = pd.read_csv(lock_meas + '\\' + meas_date + 'IMU.txt',
                               sep='\t',
                               decimal=',',
                               names=['time', 'acc0', 'acc1', 'acc2', 'gyro0', 'gyro1', 'gyro2', 'mag0', 'mag1', 'mag2'])

# Load the MoCap csv data file
mocap_data = pd.read_csv(lock_meas + '\\' + meas_date + 'MoCap.txt',
                                 sep='\t',
                                 decimal=',',
                                 names=['time', 'x', 'y', 'z', 'tracked', 'beta', 'Qx', 'Qy', 'Qz', 'Qw'])

# Merge the two data file to synronise them. In both dataset there are some data row that can't be matched, this data will
# be trown away
data = pd.merge(imu_data, mocap_data, on=['time'], how='inner')

# ===== Filters ======

# When the magneto sensors values are [0, 0, 0] that is a false value. These rows are deleted.
# When tacked is 0, it is indicates that the MoCap data is invalid. These rows are deleted too.
df = data[~(data[['mag0','mag1','mag2','tracked']] == 0).any(axis=1)]
# From now on the tracked column can be deleted, because it contains only 1s.
del df['tracked']
# Add to more columns to make a fluent orientation function (orient) and a delta time (deltat)
df = df.assign(orient=0)
df = df.assign(deltat=0)
df = df.assign(level=0)
# Convert the data frame into numpy array
dfarray = np.array(df)
# Create a lookup table to make the use of this matrix more readable.
time   = 0
acc0   = 1
acc1   = 2
acc2   = 3
gyro0  = 4
gyro1  = 5
gyro2  = 6
mag0   = 7
mag1   = 8
mag2   = 9
x      = 10
y      = 11
z      = 12
beta   = 13
Qx     = 14
Qy     = 15
Qz     = 16
Qw     = 17
orient = 18
deltat = 19
level  = 20

# Calculate the delta time between two valid measurements
for i in range(dfarray.shape[0]):
    if (i-1)>-1:
        dfarray[i,deltat] = dfarray[i,time]-dfarray[i-1,time]

# Creating a more fluent orientation function
level = 0
for i in range(dfarray.shape[0]):
    if (i-1)>-1:
        if (dfarray[i,beta]-dfarray[i-1,beta])>355:
            level = level - 1
        if (dfarray[i,beta]-dfarray[i-1,beta])<-355:
            level = level + 1
        else:
            level = level   
        dfarray[i, orient] = level*360 + dfarray[i, beta]

# NN setup

In [11]:
#1D convolution
window_size = 20
filter_length = 5
nb_filter = 4

#Train config
epochs = 50
batch_size = 16
validation_split = 0.2

# Train NN

Test results:

* First plot: x position (predictions: green, target: red)

* Second plot: z position (predictions: blue, target: yellow)

* Third plot: route of the robot on the x-z plane (predictions: blue, target: orange)

In [12]:
evalute_timeseries(dfarray,orient,window_size,filter_length,nb_filter,epochs)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 16, 4)             204       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 8, 4)              0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 4, 4)              84        
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2, 4)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 27        
Total params: 315
Trainable params: 315
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_2 to have shape (3,) but got array with shape (8,)

Original measurement on x-z plane:

In [ ]:
plt.plot(dfarray[:,-9],dfarray[:,-7])